In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("yarn").config('spark.executor.instances','100').appName("chiang").getOrCreate()

# join

In [3]:
%time df_1 = spark.read.csv("/user/spark/share/output/table_2/cross/cross_1",header=True)
%time df_2 = spark.read.csv("/user/spark/share/output/table_2/cross/cross_2",header=True)
%time df_3 = spark.read.csv("/user/spark/share/output/table_2/cross/cross_3",header=True)

CPU times: user 2.61 ms, sys: 845 µs, total: 3.45 ms
Wall time: 6.17 s
CPU times: user 0 ns, sys: 2.57 ms, total: 2.57 ms
Wall time: 3.66 s
CPU times: user 2.19 ms, sys: 110 µs, total: 2.3 ms
Wall time: 2.16 s


In [4]:
df_1.select('user_cate').distinct().count()

991866

In [7]:
df_1.count()

991866

In [5]:
%time df_2.select('user_cate').distinct().count()

CPU times: user 2.34 ms, sys: 301 µs, total: 2.64 ms
Wall time: 3.61 s


991759

In [6]:
df_3.select('user_cate').distinct().count()

992840

In [4]:
df_2 = df_2.selectExpr("user_cate as user_cate2","buy_2 as buy_2","cart_2 as cart_2","fav_2 as fav_2","pv_2 as pv_2" )

In [5]:
df_3 = df_3.selectExpr("user_cate as user_cate3","buy_3 as buy_3","cart_3 as cart_3","fav_3 as fav_3","pv_3 as pv_3" )

In [6]:
df_2.columns
# df_2.columns
# df_3.columns

['user_cate2', 'buy_2', 'cart_2', 'fav_2', 'pv_2']

In [19]:
crossall = df_1.select('user_cate').union(df_2.select('user_cate2')).distinct()
#.union(df_3.select('user_cate3')).distinct()

In [20]:
crossall = crossall.select('user_cate').union(df_3.select('user_cate3')).distinct()

In [17]:
crossall.show()

+------------+
|   user_cate|
+------------+
| 139343_4281|
| 770816_6405|
| 906336_6065|
|1111063_6240|
|  62261_4313|
| 742097_7653|
| 916126_4284|
| 168474_1243|
|  557483_806|
| 527691_1665|
| 116519_5944|
| 317210_4292|
| 501824_6427|
|  12575_1081|
|  79290_4316|
| 291082_6554|
| 969692_6421|
|  83809_7086|
| 904717_6408|
| 945231_8964|
+------------+
only showing top 20 rows



In [21]:
crossall.count()

2361188

In [7]:
crossall = df_1.join(df_2,(df_1.user_cate == df_2.user_cate2) ,'outer')

In [8]:
crossall.show()

+------------+-----+------+-----+----+------------+-----+------+-----+----+
|   user_cate|buy_1|cart_1|fav_1|pv_1|  user_cate2|buy_2|cart_2|fav_2|pv_2|
+------------+-----+------+-----+----+------------+-----+------+-----+----+
|        null| null|  null| null|null|1000053_4284|    0|     0|    0|   5|
|1000150_7208|    0|     0|    0|   4|        null| null|  null| null|null|
|        null| null|  null| null|null|1000167_1139|    0|     0|    0|  38|
|1000186_4556|    0|     0|    0|   1|        null| null|  null| null|null|
|        null| null|  null| null|null|1000215_6290|    0|     0|    0|   5|
|1000762_7604|    0|     0|    0|   2|        null| null|  null| null|null|
|        null| null|  null| null|null|1000818_6251|    0|     0|    0|   3|
| 1000901_431|    0|     0|    2|   0|        null| null|  null| null|null|
|1000920_2187|    0|     0|    0|   8|        null| null|  null| null|null|
|1001030_6540|    0|     1|    0|   0|        null| null|  null| null|null|
| 100111_428

In [9]:
crossall.count()

1714827

In [11]:
# crossall.createOrReplaceTempView('crossall')

In [17]:
# cross_nn = spark.sql("select user_cate, user_cate2, buy_1, cart_1, fav_1, pv_1, buy_2, cart_2, fav_2, pv_2 from crossall")

In [20]:
# crossall_pd= crossall.toPandas()

In [38]:
crossall_pd= crossall.toPandas()

In [82]:
len(crossall_pd)

1714827

In [62]:
crossalldf = crossall_pd.fillna({'user_cate':crossall_pd.user_cate2})
crossalldf

,user_cate,buy_1,cart_1,fav_1,pv_1,user_cate2,buy_2,cart_2,fav_2,pv_2
0,1000053_4284,None,None,None,None,1000053_4284,0,0,0,5
1,1000150_7208,0,0,0,4,None,None,None,None,None
2,1000167_1139,None,None,None,None,1000167_1139,0,0,0,38
3,1000186_4556,0,0,0,1,None,None,None,None,None
4,1000215_6290,None,None,None,None,1000215_6290,0,0,0,5
5,1000762_7604,0,0,0,2,None,None,None,None,None
6,1000818_6251,None,None,None,None,1000818_6251,0,0,0,3
7,1000901_431,0,0,2,0,None,None,None,None,None
8,1000920_2187,0,0,0,8,None,None,None,None,None
9,1001030_6540,0,1,0,0,None,None,None,None,None


In [75]:
crossalldf.drop(["user_cate2"],axis=1).to_csv("join_1.csv")

In [77]:
%time crossall_spark = spark.read.csv("/user/spark/share/output/join_1.csv",header=True)


CPU times: user 10.3 ms, sys: 119 µs, total: 10.4 ms
Wall time: 22.8 s


In [78]:
crossall_spark.show()

+---+------------+-----+------+-----+----+-----+------+-----+----+
|_c0|   user_cate|buy_1|cart_1|fav_1|pv_1|buy_2|cart_2|fav_2|pv_2|
+---+------------+-----+------+-----+----+-----+------+-----+----+
|  0|1000053_4284| null|  null| null|null|    0|     0|    0|   5|
|  1|1000150_7208|    0|     0|    0|   4| null|  null| null|null|
|  2|1000167_1139| null|  null| null|null|    0|     0|    0|  38|
|  3|1000186_4556|    0|     0|    0|   1| null|  null| null|null|
|  4|1000215_6290| null|  null| null|null|    0|     0|    0|   5|
|  5|1000762_7604|    0|     0|    0|   2| null|  null| null|null|
|  6|1000818_6251| null|  null| null|null|    0|     0|    0|   3|
|  7| 1000901_431|    0|     0|    2|   0| null|  null| null|null|
|  8|1000920_2187|    0|     0|    0|   8| null|  null| null|null|
|  9|1001030_6540|    0|     1|    0|   0| null|  null| null|null|
| 10| 100111_4281|    0|     0|    0|   1| null|  null| null|null|
| 11|1001191_6261|    0|     1|    0|   0| null|  null| null|n

In [79]:
crossall_spark.count()

1714827

In [57]:
crossall_spark = crossall_spark.join(df_3,(crossall.user_cate == df_3.user_cate3) ,'outer')

AnalysisException: 'Resolved attribute(s) user_cate#286 missing from pv_2#626,fav_1#620,user_cate#617,fav_2#625,buy_2#623,fav_3#503,cart_1#619,cart_3#502,buy_1#618,pv_1#621,user_cate2#622,pv_3#504,buy_3#501,cart_2#624,user_cate3#500 in operator !Join FullOuter, (user_cate#286 = user_cate3#500). Attribute(s) with the same name appear in the operation: user_cate. Please check if the right attribute(s) are used.;;\n!Join FullOuter, (user_cate#286 = user_cate3#500)\n:- LogicalRDD [user_cate#617, buy_1#618, cart_1#619, fav_1#620, pv_1#621, user_cate2#622, buy_2#623, cart_2#624, fav_2#625, pv_2#626], false\n+- Project [user_cate#326 AS user_cate3#500, buy_3#327 AS buy_3#501, cart_3#328 AS cart_3#502, fav_3#329 AS fav_3#503, pv_3#330 AS pv_3#504]\n   +- Relation[user_cate#326,buy_3#327,cart_3#328,fav_3#329,pv_3#330] csv\n'

In [42]:
crossall.show()

+------------+-----+------+-----+----+------------+-----+------+-----+----+------------+-----+------+-----+----+
|   user_cate|buy_1|cart_1|fav_1|pv_1|  user_cate2|buy_2|cart_2|fav_2|pv_2|  user_cate3|buy_3|cart_3|fav_3|pv_3|
+------------+-----+------+-----+----+------------+-----+------+-----+----+------------+-----+------+-----+----+
|        null| null|  null| null|null|        null| null|  null| null|null|1000053_4284|    0|     0|    0|   5|
|1000150_7208|    0|     0|    0|   4|        null| null|  null| null|null|        null| null|  null| null|null|
|1000186_4556|    0|     0|    0|   1|        null| null|  null| null|null|        null| null|  null| null|null|
|        null| null|  null| null|null|        null| null|  null| null|null|1000743_4520|    0|     0|    0|  35|
|1000762_7604|    0|     0|    0|   2|        null| null|  null| null|null|        null| null|  null| null|null|
|        null| null|  null| null|null|        null| null|  null| null|null| 100083_4280|    0|  

In [ ]:
crossall.count()

In [ ]:
crossall.show()